In [8]:
class Resistor:
    def __init__(self, ohms):
        self._ohms = ohms
        self.voltage = 0
        self.current = 0

class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
    
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'{ohms} ohms must be > 0')
        self._ohms = ohms
        # call setter for resistance and current
        self.voltage = self.current / self._ohms
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self._ohms

In [9]:
r2 = VoltageResistance(1E3)
r2.voltage = 100
print(r2.voltage)
print(r2.current)
r2.ohms = 10
print(r2.voltage)
print(r2.current)

100
0.1
0.01
0.001


In [10]:
from datetime import datetime, timedelta
class Bucket:
    def __init__(self, period) -> None:
            self.period_delta = timedelta(seconds=period)
            self.reset_time = datetime.now()
            self.quota = 0
    def __repr__(self) -> str:
          return f'Bucket(quota={self.quota})'
    

In [11]:
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0 
        bucket.reset_time = now
    bucket.quota += amount

def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

In [12]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [13]:
deduct(bucket, 99)

True

In [14]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
    
    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}', f'quota_consumed={self.quota_consumed}')
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            self.quota_consumed = 0
            self.max_quota = 0 
        elif delta < 0:
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta
    

In [15]:
class HomeWork:
    def __init__(self):
        self.__grade = 0
    
    @property
    def grade(self):
        return self.__grade

    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('grade must be between 0 and 100')
        self.__grade = value


class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
        self.new = 10
    
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('grade must be between 0 and 100')
    
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value
    
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value

In [16]:
Exam.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Exam.__init__(self)>,
              '_check_grade': <staticmethod(<function Exam._check_grade at 0x7f3b34090040>)>,
              'writing_grade': <property at 0x7f3b1eb69cb0>,
              'math_grade': <property at 0x7f3b1eb69d50>,
              '__dict__': <attribute '__dict__' of 'Exam' objects>,
              '__weakref__': <attribute '__weakref__' of 'Exam' objects>,
              '__doc__': None})

In [17]:
c = Exam()
c.__dict__

{'_writing_grade': 0, '_math_grade': 0, 'new': 10}

In [18]:
class Grade:
    def __get__(self, instance, instance_type):
        ...
    
    def __set__(self, instance, value):
        ...

In [19]:
Exam.__dict__['writing_grade'].__set__(c, 40)
# Exam.__dict__['writing_grade'].__get__(c, Exam)

In [20]:
from weakref import WeakKeyDictionary


class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()
    
    def __get__(self, instance, klass):
        # print(instance, klass)
        if instance is None:
            # this happens when we call Exam.writing_grade
            return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('grade must be between 0 and 100')
        self._values[instance] = value

In [21]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    

In [22]:
first_exam = Exam()
first_exam.writing_grade = 40
first_exam.science_grade = 50
# print(f'Writing {first_exam.writing_grade}, Science {first_exam.science_grade}')

second_exam = Exam()
second_exam.writing_grade = 55
second_exam.science_grade = 60
print(f'First {first_exam.writing_grade}, Second {second_exam.writing_grade}')
print(f'Writing {first_exam.science_grade}, Science {second_exam.science_grade}')

Exam.__dict__['writing_grade'].__get__(first_exam, Exam)

First 40, Second 55
Writing 50, Science 60


40

In [23]:
dict(Exam.science_grade._values)

{<__main__.Exam at 0x7f3b1eb55e50>: 50, <__main__.Exam at 0x7f3b34086c50>: 60}

In [24]:
import weakref

class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"MyClass({self.name})"

obj1 = MyClass("Object 1")
obj2 = MyClass("Object 2")

d = weakref.WeakKeyDictionary()
d[obj1] = "Value for Object 1"

print(dict(d))
# Output: {MyClass(Object 1): 'Value for Object 1'}

del obj1

print(dict(d))
# Output: {}


{MyClass(Object 1): 'Value for Object 1'}
{}


In [25]:
class LazyRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattr__(self, name):
        print("Inside __getattr__")
        value = f'Value for {name}'
        setattr(self, name, value)
        return value

    def __getattribute__(self, __name: str):
        print("Inside __getattribute__")
        return super().__getattribute__(__name)

In [26]:
c = LazyRecord()
d = LazyRecord()
print(c.foo)
print(d.foo)
setattr(c, 'bew', 'bew')

Inside __getattribute__
Inside __getattr__
Value for foo
Inside __getattribute__
Inside __getattr__
Value for foo


In [27]:
class LoggingLazyRecord(LazyRecord):
    # def __getattr__(self, name):
    #     print(f"** Called __getattr__({name}!r), populting instance dictionary")
    #     # result = super().__getattr__(name)
    #     result = f"Value for {name}"
    #     print(f"** Returning {result!r}")
    #     return result
    pass

data = LoggingLazyRecord()
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ', data.__dict__)

Inside __getattribute__
Before: {'exists': 5}
Inside __getattribute__
Inside __getattr__
foo:    Value for foo
Inside __getattribute__
After:  {'exists': 5, 'foo': 'Value for foo'}


In [28]:
class BrokenDictonary:
    def __init__(self, data) -> None:
        self._data = data
    
    def __getattribute__(self, __name: str):
        # print("Here Here")
        print(self._data)
        return self._data[__name]

In [29]:
data = BrokenDictonary({'a' : 3})
data.a

RecursionError: maximum recursion depth exceeded

In [ ]:
import pprint
from functools import partial
print = partial(pprint.pprint, width=60, indent=2)

class LoggingSavingRecord:
    def __setattr__(self, name, value):
        print(f"** Called __setattr__({name!r}, {value!r})")
        super().__setattr__(name, value)


data = LoggingSavingRecord()
print(data.__dict__)
data.foo = 5
l = {'a' : {'a' : { 'a' : 3}}}
print(data.__dict__)
data.foo = 7
print(data.__dict__)


In [ ]:
from typing import Any


class DictionaryRecord:
    def __init__(self):
        print("__init__ called")
        self._data = {}
    
    def __getattribute__(self, __name: str) -> Any:
        print("__getattribute__ called")
        data_dict = super().__getattribute__('_data')
        return data_dict[__name]

    # def __setattr__(self, name: str, value: Any) -> None:
    #     print(f"__setattr__ called {name}, {value}")
    #     if name != '_data':
    #         self._data[name] = value
    #     else:
    #         super().__setattr__(name, value)


data = DictionaryRecord()
data.foo = 10
print(data.__dict__)
print(data.foo)


In [ ]:
class Meta(type):
    def __new__(cls, applied_klass, applied_klass_bases, klass__dict__):
        print((cls, applied_klass, applied_klass_bases, klass__dict__))
        return type.__new__(cls, applied_klass, applied_klass_bases, klass__dict__)


class MyClass(metaclass=Meta):
    stuff = 123

    def foo(self):
        pass

class MySubclass(MyClass):
    other = 456

    def bar(self):
        pass
    

( <class '__main__.Meta'>,
  'MyClass',
  (),
  { '__module__': '__main__',
    '__qualname__': 'MyClass',
    'foo': <function MyClass.foo at 0x7f3241307060>,
    'stuff': 123})
( <class '__main__.Meta'>,
  'MySubclass',
  (<class '__main__.MyClass'>,),
  { '__module__': '__main__',
    '__qualname__': 'MySubclass',
    'bar': <function MySubclass.bar at 0x7f32410cd120>,
    'other': 456})


In [ ]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        if (
            not class_dict.get("is_root") 
            and bases 
            and class_dict.get("sides", 0) < 3
        ):
            raise ValueError("Polygons need at least 3 sides")
        return type.__new__(meta, name, bases, class_dict)


class ValidateFilledPolygon(ValidatePolygon):
    def __new__(meta, name, bases, class_dict):
        if (
            not class_dict.get("is_root")
            and bases
            and class_dict.get("color") not in ("red", "green", "blue")
        ):
            raise ValueError("Fill color must be supported.")
        return type.__new__(meta, name, bases, class_dict)


class Polygon(metaclass=ValidateFilledPolygon):
    sides = None
    colors = None

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180


class FilledPolygon(Polygon, metaclass=ValidateFilledPolygon):
    is_root = True
    color = None


class Triangle(FilledPolygon):
    sides = 3
    color = "red"


assert Triangle.interior_angles() == 180


In [ ]:
class BetterPolygonClass:
    def __init_subclass__(drived_cls):
        super.__init_subclass__()
        print(drived_cls)
        if drived_cls.sides < 3:
            raise ValueError('Polygons need at least 3 sides')

class Hexagon(Polygon, BetterPolygonClass):
    sides = 2
    color= "red"

<class '__main__.Hexagon'>


ValueError: Polygons need at least 3 sides

In [ ]:
class Top:
    def __init_subclass__(cls) -> None:
        super().__init_subclass__()
        print("Top.__init_subclass__ called")

class Left(Top):
    def __init_subclass__(cls) -> None:
        return super().__init_subclass__()
        print("Left.__init_subclass__ called")


class Right(Top):
    def __init_subclass__(cls) -> None:
        return super().__init_subclass__()
        print("Right.__init_subclass__ called")

class Bottom(Left, Right):
    def __init_subclass__(cls) -> None:
        return super().__init_subclass__()
    print("Bottom __init_subclass__ called")

Top.__init_subclass__ called
Top.__init_subclass__ called
Bottom __init_subclass__ called
Top.__init_subclass__ called


In [ ]:
import json

class Serializable:
    def __init__(self, *args):
        self.args = args
    def serialize(self):
        return json.dumps({"args": self.args})


In [ ]:
class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

c = Point2D(5, 3)
c.serialize()

class Deserializable:
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params["args"])

In [32]:
import json 

class BetterSerializable:
    def __init__(self, *args):
        self.args = args
    
    def serialize(self):
        return json.dumps({
            "class": self.__class__.__name__,
            "args": self.args
        })


registry = {}

def register_class(target_class):
    registry[target_class.__name__] = target_class

def deserialize(data):
    params = json.loads(data)
    name = params["class"]
    target_class = registry[name]
    return target_class(*params["args"])

class EvenBetterPoint2d(BetterSerializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    

In [33]:
from dataclasses import dataclass

In [50]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print((meta, name, bases, class_dict))
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls

class RegisteredSerializable(BetterSerializable, metaclass=Meta):
    pass


@dataclass
class Vector3D(RegisteredSerializable):
    x: float
    y: float
    z: float

    def __post_init__(self):
        super().__init__(self.x, self.y, self.z)


(<class '__main__.Meta'>, 'RegisteredSerializable', (<class '__main__.BetterSerializable'>,), {'__module__': '__main__', '__qualname__': 'RegisteredSerializable'})
(<class '__main__.Meta'>, 'Vector3D', (<class '__main__.RegisteredSerializable'>,), {'__module__': '__main__', '__qualname__': 'Vector3D', '__annotations__': {'x': <class 'float'>, 'y': <class 'float'>, 'z': <class 'float'>}, '__post_init__': <function Vector3D.__post_init__ at 0x7f3b1e956660>, '__classcell__': <cell at 0x7f3b1ef541c0: empty>})


In [51]:
before = Vector3D(10, -7, 3.14)
print("Before:", before)
data = before.serialize()
print(data)

Before: Vector3D(x=10, y=-7, z=3.14)
{"class": "Vector3D", "args": [10, -7, 3.14]}
